In [36]:
import pandas as pd
import numpy as np
import sqlalchemy as db
import os 

Connection to MySQL

In [37]:
conn = pd.read_json('connection.json', typ='series')
host = conn['host']
user = conn['user']
password = conn['password']
port = conn['port']

db_name = input('DB Name: ')
engine = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port + '/' + db_name)

DB Name: PALB777


Fine Query

In [ ]:
limit = 100_000
offset = 0
plane = 'rp7779'
year = 2022
df = []

while True:
    try:
        data = pd.read_sql(f"SELECT * FROM `{plane}-{year}` LIMIT {limit} OFFSET {offset}", engine)
        planeIDs = list(data['ID'].unique())
        dates = list(data['Date'].unique())
        minIndex = min(data['index'])
        maxIndex = max(data['index'])
        rm = list(data['RM'].unique())
        acft = list(data['Plane'].unique())
        data.to_parquet(f"{db_name}/{plane}-{year}/{offset}_{offset + limit}.parquet")
        print(f"ID: {planeIDs}, Dates: {dates}, IndexRange: ({minIndex, maxIndex})")
        print(f"Aircraft Included: {acft}, Registration Marking: {rm}")
        print(f"Index Counter: {offset} - {offset + limit}")
        print("-------------------------------------------------------------------")
        offset += limit
    except Exception as e:
        print(f"AN ERROR OCCURED: {e}")
        break

Reading Data

In [30]:
directory = 'C:/Users/Kyle Carlo C. Lasala/Desktop/PAL/repo/Predicting-Fuel-Consumption/PALB777/rp7778-2022'

test = os.listdir(directory)

In [33]:
np.array(test)

array(['0_100000.parquet', '10000000_10100000.parquet',
       '1000000_1100000.parquet', '100000_200000.parquet',
       '10100000_10200000.parquet', '10200000_10300000.parquet',
       '10300000_10400000.parquet', '10400000_10500000.parquet',
       '10500000_10600000.parquet', '10600000_10700000.parquet',
       '10700000_10800000.parquet', '10800000_10900000.parquet',
       '10900000_11000000.parquet', '11000000_11100000.parquet',
       '1100000_1200000.parquet', '11100000_11200000.parquet',
       '11200000_11300000.parquet', '11300000_11400000.parquet',
       '11400000_11500000.parquet', '11500000_11600000.parquet',
       '11600000_11700000.parquet', '11700000_11800000.parquet',
       '11800000_11900000.parquet', '11900000_12000000.parquet',
       '12000000_12100000.parquet', '1200000_1300000.parquet',
       '12100000_12200000.parquet', '12200000_12300000.parquet',
       '12300000_12400000.parquet', '12400000_12500000.parquet',
       '12500000_12600000.parquet', '1260000

In [35]:
np.sort(test)

array(['0_100000.parquet', '10000000_10100000.parquet',
       '1000000_1100000.parquet', '100000_200000.parquet',
       '10100000_10200000.parquet', '10200000_10300000.parquet',
       '10300000_10400000.parquet', '10400000_10500000.parquet',
       '10500000_10600000.parquet', '10600000_10700000.parquet',
       '10700000_10800000.parquet', '10800000_10900000.parquet',
       '10900000_11000000.parquet', '11000000_11100000.parquet',
       '1100000_1200000.parquet', '11100000_11200000.parquet',
       '11200000_11300000.parquet', '11300000_11400000.parquet',
       '11400000_11500000.parquet', '11500000_11600000.parquet',
       '11600000_11700000.parquet', '11700000_11800000.parquet',
       '11800000_11900000.parquet', '11900000_12000000.parquet',
       '12000000_12100000.parquet', '1200000_1300000.parquet',
       '12100000_12200000.parquet', '12200000_12300000.parquet',
       '12300000_12400000.parquet', '12400000_12500000.parquet',
       '12500000_12600000.parquet', '1260000